## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
filtered_df = application_df.drop(columns=['EIN', 'NAME'])
filtered_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
filtered_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
filtered_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [27]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    filtered_df['APPLICATION_TYPE'] = filtered_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
filtered_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Look at CLASSIFICATION value counts for binning
classifications = filtered_df['CLASSIFICATION'].value_counts()
cls_df = pd.DataFrame(classifications)
cls_df

,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
Other,1484
C7000,777


In [29]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for idx, cls in cls_df.iterrows():
    if cls[0] < 500:
        classifications_to_replace.append(idx)

# Replace in dataframe
for cls in classifications_to_replace:
    filtered_df['CLASSIFICATION'] = filtered_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(filtered_df)
dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL']
X = dummies_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5)                 225       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6118 - accuracy: 0.6900: 0s - loss: 0.6302 - ac
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5663 - accuracy: 0.7252
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5590 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5562 - accuracy: 0.7298
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7299
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7296
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5528 - accuracy: 0.7296
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7299
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5517 - accuracy: 0.7306
Epoch 10/100
804/804 [==============================] - 1s 1ms/step

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5538 - accuracy: 0.7263
Loss: 0.5538271069526672, Accuracy: 0.7262973785400391


In [17]:
# Export our model to HDF5 file
import os
checkpoint_path = "Output/AlphabetSoupCharity.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 804
cp_callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_frequency = 5*batch_size
)

nn.save_weights(checkpoint_path.format(epoch=0))

fit_model = nn.fit(X_train_scaled,y_train,
    epochs=100,
    batch_size=batch_size,
    callbacks=[cp_callbacks],
    validation_data=(X_test,y_test),
    verbose=0
    )


Epoch 00001: saving model to Output\AlphabetSoupCharity.h5

Epoch 00002: saving model to Output\AlphabetSoupCharity.h5

Epoch 00003: saving model to Output\AlphabetSoupCharity.h5

Epoch 00004: saving model to Output\AlphabetSoupCharity.h5

Epoch 00005: saving model to Output\AlphabetSoupCharity.h5

Epoch 00006: saving model to Output\AlphabetSoupCharity.h5

Epoch 00007: saving model to Output\AlphabetSoupCharity.h5

Epoch 00008: saving model to Output\AlphabetSoupCharity.h5

Epoch 00009: saving model to Output\AlphabetSoupCharity.h5

Epoch 00010: saving model to Output\AlphabetSoupCharity.h5

Epoch 00011: saving model to Output\AlphabetSoupCharity.h5

Epoch 00012: saving model to Output\AlphabetSoupCharity.h5

Epoch 00013: saving model to Output\AlphabetSoupCharity.h5

Epoch 00014: saving model to Output\AlphabetSoupCharity.h5

Epoch 00015: saving model to Output\AlphabetSoupCharity.h5

Epoch 00016: saving model to Output\AlphabetSoupCharity.h5

Epoch 00017: saving model to Output\Alp

In [18]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5520 - accuracy: 0.7290
Loss: 0.5520367622375488, Accuracy: 0.7289795875549316
